In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

In [4]:
"""
Computes the following chain of quantities used in the paper/discussion:
1) alpha = G M mu / c^2 (dimensionless; we compute as r_g * mu in 1/m units)
2) E_obs ≈ 0.3 * eps * alpha^(7/2) * Lambda (V/m) -- (paper Eq.4 style)
3) a0 = e E_obs / (m_e omega), gamma = sqrt(1 + a0^2)
4) suppression S = gamma * (mu / omega_p)^2 (collisionless regime)

Notes:
- mu and omega_p are in eV for the suppression ratio.
- All SI conversions are handled internally.
"""

from typing import Tuple, Dict, Iterable
import numpy as np

# Physical constants (SI)
G = 6.67430e-11 # gravitational constant, m^3 kg^-1 s^-2
c = 299792458 # speed of light, m/s
hbar = 1.054571817e-34 # reduced Planck constant, J·s
e_charge = 1.602176634e-19 # elementary charge, C (also J/eV)
m_e = 9.10938356e-31 # electron mass, kg

# Conversion factor: hbar * c in eV * m
HBAR_C_eVm = hbar * c / e_charge

# Utility/conversion functions
def omega_p_eV_from_ne_cm3(n_e_cm3: float) -> float:
    """
    Parameters
    - n_e_cm3 (float): electron density in cm^-3

    Returns
    - omega_p_eV (float): plasma frequency in eV
    """
    return 3.713e-11 * np.sqrt(n_e_cm3)


def mu_eV_to_inverse_m(mu_eV: float) -> float:
    """
    Parameters
    - mu_eV (float): mass / frequency in eV

    Returns
    - mu_1m (float): mass / frequency in inverse meters (1/m)
    """
    return mu_eV / HBAR_C_eVm


def gravitational_radius(M_kg: float) -> float:
    """
    Parameters
    - M_kg (float): mass in kg

    Returns
    - r_g (float): gravitational radius in meters
    """
    return G * M_kg / c**2


def alpha_from_M_mu(M_kg: float, mu_eV: float) -> float:
    """
    Parameters
    - M_kg (float): mass in kg
    - mu_eV (float): mass / frequency in eV

    Returns
    - alpha (float): dimensionless coupling α = G M μ
    """
    r_g = gravitational_radius(M_kg)
    mu_1m = mu_eV_to_inverse_m(mu_eV)
    return r_g * mu_1m

# Physics formulas
def E_obs_from_eps_alpha(eps: float, alpha: float, Lambda: float) -> float:
    """    
    Parameters
    - eps (float): kinetic mixing (dimensionless)
    - alpha (float): dimensionless coupling
    - Lambda (float): scale in V/m

    Returns
    - E_obs (float): observed electric field in V/m
    """
    return 0.3 * eps * alpha**(7 / 2) * Lambda


def gamma_from_E_and_mu(E_Vm: float, mu_eV: float) -> Tuple[float, float]:
    """
    Parameters
    - E_Vm (float): electric field in V/m
    - mu_eV (float): mass / frequency in eV

    Returns
    - a0 (float): quiver parameter (dimensionless)
    - gamma (float): Lorentz boost (dimensionless)
    """
    omega_rad = mu_eV * e_charge / hbar # angular frequency in rad/s
    a0 = (e_charge * E_Vm) / (m_e * omega_rad)
    gamma = np.sqrt(1.0 + a0**2)
    return a0, gamma


def suppression_S(gamma: float, mu_eV: float, omega_p_eV: float) -> float:
    """
    Parameters
    - gamma (float): Lorentz boost (dimensionless)
    - mu_eV (float): mass / frequency
    - omega_p_eV (float): plasma frequency

    Returns
    - S (float): suppression factor (dimensionless)
    """
    return gamma * (mu_eV / omega_p_eV)**2

# Pretty-print helpers
def fmt_scientific(x: float, sig: int = 3) -> str:
    """
    Format a float in scientific notation with specified significant figures.

    Parameters
    - x (float): number to format
    - sig (int): significant figures

    Returns
    - formatted string
    """
    return f"{x:.{sig}e}"

def run_grid_and_print(
    bh_specs: Dict[str, Dict[str, float]],
    eps_values: Iterable[float],
    mu_values_eV: Iterable[float],
    n_e_values_cm3: Iterable[float],
) -> None:
    """
    Run the full grid of calculations and print results.

    Parameters
    - bh_specs (dict): black hole specifications with mass and Lambda
    - eps_values (iterable): list of kinetic mixing values
    - mu_values_eV (iterable): list of dark-photon masses in eV
    - n_e_values_cm3 (iterable): list of electron densities in cm^-3
    """

    for bh_name, params in bh_specs.items():
        M_kg = params["mass"]
        Lambda = params["Lambda"]
        print(f"Black hole: {bh_name:6s} | M = {M_kg:.4e} kg | Lambda = {Lambda:.4e} V/m")
        for mu_eV in mu_values_eV:
            alpha_val = alpha_from_M_mu(M_kg, mu_eV)
            print(f"\nmu = {fmt_scientific(mu_eV)} eV -> alpha = {fmt_scientific(alpha_val)}")
            for eps in eps_values:
                E = E_obs_from_eps_alpha(eps, alpha_val, Lambda)
                a0, gamma = gamma_from_E_and_mu(E, mu_eV)
                print(f"  eps={eps:.1e} | E_obs={fmt_scientific(E)} V/m | a0={fmt_scientific(a0)} | gamma={fmt_scientific(gamma)}")
                for n_e in n_e_values_cm3:
                    omega_p_eV = omega_p_eV_from_ne_cm3(n_e)
                    S_raw = suppression_S(gamma, mu_eV, omega_p_eV)
                    S_capped = min(S_raw, 1.0)    # do not exceed vacuum mixing
                    W = np.inf if S_capped == 0 else 1.0 / S_capped
                    print(
                        f"n_e={int(n_e):7d} cm^-3 | omega_p={fmt_scientific(omega_p_eV)} eV "
                        f"| S_raw={fmt_scientific(S_raw)} | S_capped={fmt_scientific(S_capped)} | W={fmt_scientific(W)}"
                    )

In [3]:
# Run the main function
if __name__ == "__main__":
    # BH masses (kg) used in the draft / earlier calculations
    M_sun = 1.98847e30 # kg
    bh_specs = {
        "M87": {"mass": 6.5e9 * M_sun, "Lambda": 2.2e14}, # Lambda in V/m (from draft)
        "SgrA": {"mass": 4e6 * M_sun, "Lambda": 3.6e17},
    }

    # Parameter grids
    eps_values = [1e-8] # example claimed sensitivity; can provide list to explore
    mu_values_eV = [1e-22, 1e-20, 1e-19, 1e-17, 1e-16] # dark-photon masses (eV)
    n_e_values_cm3 = [1e4, 1e5, 1e6, 1e7] # electron densities in cm^-3

    # Run and print detailed grid
    run_grid_and_print(bh_specs, eps_values, mu_values_eV, n_e_values_cm3)

Black hole: M87    | M = 1.2925e+40 kg | Lambda = 2.2000e+14 V/m

mu = 1.000e-22 eV -> alpha = 4.864e-03
  eps=1.0e-08 | E_obs=5.298e-03 V/m | a0=6.133e+15 | gamma=6.133e+15
n_e=  10000 cm^-3 | omega_p=3.713e-09 eV | S_raw=4.449e-12 | S_capped=4.449e-12 | W=2.248e+11
n_e= 100000 cm^-3 | omega_p=1.174e-08 eV | S_raw=4.449e-13 | S_capped=4.449e-13 | W=2.248e+12
n_e=1000000 cm^-3 | omega_p=3.713e-08 eV | S_raw=4.449e-14 | S_capped=4.449e-14 | W=2.248e+13
n_e=10000000 cm^-3 | omega_p=1.174e-07 eV | S_raw=4.449e-15 | S_capped=4.449e-15 | W=2.248e+14

mu = 1.000e-20 eV -> alpha = 4.864e-01
  eps=1.0e-08 | E_obs=5.298e+04 V/m | a0=6.133e+20 | gamma=6.133e+20
n_e=  10000 cm^-3 | omega_p=3.713e-09 eV | S_raw=4.449e-03 | S_capped=4.449e-03 | W=2.248e+02
n_e= 100000 cm^-3 | omega_p=1.174e-08 eV | S_raw=4.449e-04 | S_capped=4.449e-04 | W=2.248e+03
n_e=1000000 cm^-3 | omega_p=3.713e-08 eV | S_raw=4.449e-05 | S_capped=4.449e-05 | W=2.248e+04
n_e=10000000 cm^-3 | omega_p=1.174e-07 eV | S_raw=4.449e-0